In [1]:
import os
import json
import pandas as pd
import pickle
from definitions import ROOT_DIR

In [2]:
def find_word(extended_word, word_list):
    for w in word_list:
        if w in extended_word:
            return w
    return None

# Create the datasets with the performance evaluations on fixed target environments

In [3]:
env_names = ["ant", "hopper", "walker", "half_cheetah"]
algos = ("attention_no_encoding", "simple", "rma_phase_1", "attention", "encoding", "convnet", "adapted")
algo_name_dict = {
    "attention_no_encoding": "dmap-ne",
    "simple": "simple",
    "rma_phase_1": "oracle",
    "attention": "dmap",
    "encoding": "oracle",
    "convnet": "tcn",
    "adapted": "rma"
}
datasets = ("sigma_01", "sigma_03", "sigma_05", "sigma_07",
            "sigma_between_0_01", "sigma_between_0_03", "sigma_between_0_05", "sigma_between_0_07")
train_sigma_list = ("sigma_01", "sigma_03", "sigma_05")
seed_list = ("seed_0", "seed_1", "seed_2", "seed_3", "seed_4")

In [4]:
env_summary_dict = {}

for name in env_names:
    env_summary_dict[name] = []
    result_dir = os.path.join(ROOT_DIR, "data", name, "performance")
    result_dir_content = os.listdir(result_dir)
    
    result_json_list = []
    for file_name in result_dir_content:
        file_path = os.path.join(result_dir, file_name)
        if os.path.isfile(file_path):
            assert ".json" in file_name
            with open(file_path, "r") as file:
                result_json = json.load(file)
            result_json_list.append(result_json)

    for result_json in result_json_list:
        for experiment_name, data in result_json.items():
            algorithm_name = find_word(experiment_name, algos)
            assert algorithm_name is not None, experiment_name
            train_sigma = find_word(experiment_name, train_sigma_list)
            assert train_sigma is not None, experiment_name
            results = data["results"]
            for dataset_name, reward_list in results.items():
                perturbation_summary_path = os.path.join(ROOT_DIR, "data", "xmls", name, dataset_name, "perturbation_summary.pkl")
                with open(perturbation_summary_path, "rb") as file:
                    perturbation_summary = pickle.load(file)
                perturbation_names = perturbation_summary["perturbations"]
                perturbation_list = perturbation_summary["values"]
                for idx, reward in enumerate(reward_list):
                    perturbation_dict = dict(zip(perturbation_names, perturbation_list[idx]))
                    summary_dict = {
                        "env": name,
                        "algorithm": algorithm_name,
                        "train_sigma": train_sigma,
                        "test_sigma": dataset_name,
                        "xml_idx": idx,
                        "reward": reward
                    }
                    summary_dict.update(perturbation_dict)
                    env_summary_dict[name].append(summary_dict)

In [5]:
summary_df_list = []
for env_name, summary_list in env_summary_dict.items():
    summary_df = pd.DataFrame(summary_list)
    summary_df.replace("rma_phase_1", "oracle", inplace=True)
    summary_df.replace("encoding", "oracle", inplace=True)
    summary_df.replace("test_sigma_between_0_01", "sigma_01", inplace=True)
    summary_df.replace("test_sigma_between_0_03", "sigma_03", inplace=True)
    summary_df.replace("test_sigma_between_0_05", "sigma_05", inplace=True)
    summary_df.replace("test_sigma_between_0_07", "sigma_07", inplace=True)
    summary_df_list.append((env_name, summary_df))

In [6]:
# Check that there are no nans
for summary in summary_df_list:
    df = summary[1]
    print("Number of nans in df", summary[0], df.isna().any(axis=1).sum())

Number of nans in df ant 0
Number of nans in df hopper 0
Number of nans in df walker 0
Number of nans in df half_cheetah 0


In [7]:
for env_name, df in summary_df_list:
    print(env_name, df.algorithm.unique())
    for algo in df.algorithm.unique():
        for train_sigma in df.train_sigma.unique():
            for test_sigma in df.test_sigma.unique():
                small_df = df[(df.algorithm == algo) & (df.train_sigma == train_sigma) & (df.test_sigma == test_sigma)]
                if len(small_df) != 500:
                    print("env_name", env_name, "algo", algo, "train_sigma", train_sigma, "test_sigma", test_sigma, "number", len(small_df))
    df_name = f"{env_name}_experiments_df.pkl"
    out_path = os.path.join(ROOT_DIR, "data", df_name)
    df.to_pickle(out_path)

ant ['attention_no_encoding' 'convnet' 'adapted' 'attention' 'simple' 'oracle']
hopper ['oracle' 'attention_no_encoding' 'simple' 'adapted' 'convnet' 'attention']
walker ['adapted' 'simple' 'oracle' 'attention_no_encoding' 'convnet' 'attention']
half_cheetah ['adapted' 'oracle' 'attention' 'simple' 'convnet' 'attention_no_encoding']
